<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/multivariable_lstm_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro
this is my practice of [this](https://qwerty1434.github.io/%EB%8B%A4%EC%A4%91%EB%B3%80%EC%88%98%EC%9D%98-%ED%83%80%EC%9E%84%EC%8B%9C%EB%A6%AC%EC%A6%88-LSTM%EC%97%B0%EC%8A%B5)

In [2]:
import pandas as pd
import numpy as np
from pandas import read_csv
from datetime import datetime
from pandas import DataFrame
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

In [115]:
# def parse(x):
#   return datetime.strptime(x, '%Y %m %d %H')
dataset = read_csv('/content/drive/My Drive/Datasets/multivariable_timeseries.csv', parse_dates = [['year', 'month', 'day', 'hour']],
                 index_col=0)


In [116]:
dataset

,No,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
year_month_day_hour,,,,,,,,,
2010 1 1 0,1,NaN,-21,-11.0,1021.0,NW,1.79,0,0
2010 1 1 1,2,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2010 1 1 2,3,NaN,-21,-11.0,1019.0,NW,6.71,0,0
2010 1 1 3,4,NaN,-21,-14.0,1019.0,NW,9.84,0,0
2010 1 1 4,5,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...
2014 12 31 19,43820,8.0,-23,-2.0,1034.0,NW,231.97,0,0
2014 12 31 20,43821,10.0,-22,-3.0,1034.0,NW,237.78,0,0
2014 12 31 21,43822,10.0,-22,-3.0,1034.0,NW,242.70,0,0


Number 컬럼 삭제

In [117]:
dataset.drop('No', axis=1, inplace=True)

column 이름 재정의

In [118]:
dataset.columns = ['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']

In [119]:
dataset

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
year_month_day_hour,,,,,,,,
2010 1 1 0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
2010 1 1 1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2010 1 1 2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
2010 1 1 3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
2010 1 1 4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...
2014 12 31 19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
2014 12 31 20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
2014 12 31 21,10.0,-22,-3.0,1034.0,NW,242.70,0,0


In [120]:
dataset.index.name = 'date'

In [121]:
# Fill NA/NaN values using the specified method.
striped_dataset['pollution'].fillna(0, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [122]:
striped_dataset = dataset[24:]

In [123]:
striped_dataset

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010 1 2 0,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010 1 2 1,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010 1 2 2,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010 1 2 3,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010 1 2 4,138.0,-7,-5.0,1022.0,SE,6.25,2,0
...,...,...,...,...,...,...,...,...
2014 12 31 19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
2014 12 31 20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
2014 12 31 21,10.0,-22,-3.0,1034.0,NW,242.70,0,0


빈 칸을 0으로 채운다

In [124]:
striped_dataset.to_csv('pollution.csv')

# data scaling

In [125]:
dataset = read_csv('pollution.csv')
values = dataset.values

In [126]:
dataset

,date,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
0,2010 1 2 0,129.0,-16,-4.0,1020.0,SE,1.79,0,0
1,2010 1 2 1,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2,2010 1 2 2,159.0,-11,-5.0,1021.0,SE,3.57,0,0
3,2010 1 2 3,181.0,-7,-5.0,1022.0,SE,5.36,1,0
4,2010 1 2 4,138.0,-7,-5.0,1022.0,SE,6.25,2,0
...,...,...,...,...,...,...,...,...,...
43795,2014 12 31 19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43796,2014 12 31 20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43797,2014 12 31 21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43798,2014 12 31 22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [127]:
encoder = preprocessing.LabelEncoder()
values[:,6] = encoder.fit_transform(values[:,6]) # wind_dir 변수를 숫자로 인코딩함

In [129]:
values[:,2] = values[:,2].astype('float32')

In [130]:
values

array([['2010 1 2 0', 129.0, -16.0, ..., 6, 0, 0],
       ['2010 1 2 1', 148.0, -15.0, ..., 12, 0, 0],
       ['2010 1 2 2', 159.0, -11.0, ..., 17, 0, 0],
       ...,
       ['2014 12 31 21', 10.0, -22.0, ..., 2250, 0, 0],
       ['2014 12 31 22', 8.0, -22.0, ..., 2272, 0, 0],
       ['2014 12 31 23', 12.0, -21.0, ..., 2286, 0, 0]], dtype=object)

# scaler
모든 값을 0~1 사이로 정규화

In [131]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(values)

ValueError: ignored